In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [11]:
df=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

df1=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
print(df.shape)
print(df1.shape)

x_train = df.values[:,1:785]
y_train = df.values[:, 0]
x_test = df1.values[:, 0:784]
x_train.shape


(42000, 785)
(28000, 784)


(42000, 784)

In [13]:
x_train=x_train/255.0
x_train
x_test=x_test/255.0

In [14]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape)
print(x_test.shape)


(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [15]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Dropout
from keras import regularizers
from keras import metrics
from keras.utils.np_utils import to_categorical
from keras import optimizers
from scipy import misc


In [16]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid=train_test_split(x_train,y_train,test_size=0.25)
y_train = to_categorical(y_train,10)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
print(x_valid.shape,y_valid.shape,sep='\n')

(10500, 28, 28, 1)
(10500,)


In [18]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(4,4),activation='relu',data_format='channels_last',input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1)))
model.add(Dropout(rate=0.3,seed=3))
model.add(Conv2D(32,kernel_size=(4,4),padding='valid', activation='relu'))
model.add(Conv2D(32,kernel_size=(2,2),padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4), strides=(1,1)))
model.add(Dropout(rate=0.3,seed=8))
model.add(Flatten())
model.add(Dense(512,activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(10, activation = 'sigmoid'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.categorical_accuracy])


In [20]:
model.fit(x_train,y_train,epochs=30,batch_size=64)
model.save('mnist_model.h5')
model.save_weights('mnist_weights.h5')

Epoch 1/20
493/493 [==============================] - 73s 146ms/step - loss: 0.5521 - categorical_accuracy: 0.8228
Epoch 2/20
493/493 [==============================] - 71s 144ms/step - loss: 0.0629 - categorical_accuracy: 0.9813
Epoch 3/20
493/493 [==============================] - 71s 144ms/step - loss: 0.0460 - categorical_accuracy: 0.9858
Epoch 4/20
493/493 [==============================] - 71s 144ms/step - loss: 0.0351 - categorical_accuracy: 0.9893
Epoch 5/20
493/493 [==============================] - 71s 145ms/step - loss: 0.0292 - categorical_accuracy: 0.9911
Epoch 6/20
493/493 [==============================] - 71s 145ms/step - loss: 0.0251 - categorical_accuracy: 0.9919
Epoch 7/20
493/493 [==============================] - 70s 143ms/step - loss: 0.0198 - categorical_accuracy: 0.9943
Epoch 8/20
493/493 [==============================] - 71s 143ms/step - loss: 0.0181 - categorical_accuracy: 0.9942
Epoch 9/20
493/493 [==============================] - 71s 144ms/step - loss: 0.0

acc=model.evaluate(x_valid,y_valid)
print(acc)

In [24]:
testing=model.predict_classes(x_test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [25]:
res=pd.DataFrame({"ImageId": list(range(1,len(testing)+1)),"Label": testing})

In [26]:
res.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [27]:
res.to_csv("digit_reco.csv",index=False,header=True)

In [28]:
from IPython.display import FileLink
FileLink(r'digit_reco.csv')

/kaggle/working/digit_reco.csv